In [40]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.io import wavfile
from numpy.linalg import svd
from scipy.stats.mstats import gmean
from matplotlib import rcParams
import scipy
import os
import sys
import glob
import pickle
from music21 import converter, instrument, note, chord, stream
from keras import backend as K
from keras import Input
from keras.models import Sequential
from keras.models import Model
from keras.layers import Embedding
from keras.layers import Concatenate
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Permute
from keras.layers import RepeatVector
from keras.layers import Multiply
from keras.layers import Lambda
from keras.optimizers import RMSprop
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [5]:
def create_network(notes_in, durations_in, n_notes, n_durations):
# def create_network(network_input, n_vocab):
#     model = Sequential()
#     model.add(LSTM(
#         512,
#         input_shape=(network_input.shape[1], network_input.shape[2]),
#         recurrent_dropout=0.3,
#         return_sequences=True
#     ))
    
#     ''' Your Code Here '''
#     #LSTM layer
#     model.add(LSTM(
#         512,
#         recurrent_dropout=0.3
#     ))
#     #batch normalization layer
#     #model.add(BatchNormalization())
#     #dropout layer
#     model.add(Dropout(0.3))
#     #fully connected layer
#     model.add(Dense(256))
#     #relu
#     model.add(Activation('relu'))
#     #batch normalization layer
#     model.add(BatchNorm())
#     #dropout layer
#     model.add(Dropout(0.3))
#     #fully connected layer
#     model.add(Dense(n_vocab)) #fix this one
#     #temperature
#     model.add(Lambda(lambda x: x/ .6))
#     #softmax
#     model.add(Activation('softmax'))
    
#     model.compile(loss="categorical_crossentropy", optimizer='adam')
#     #model.fit(network_input, epochs=50, verbose=0)
    
#     return model

    batch_size = 64
    embedding_dim = 100 #256
    rnn_units = 256
    x1 = Embedding(n_notes, embedding_dim)(notes_in)
    x2 = Embedding(n_durations, embedding_dim)(durations_in)

    x = Concatenate()([x1,x2])

    x = LSTM(rnn_units, return_sequences=True)(x)
    x = LSTM(rnn_units, return_sequences=True)(x)

    e = Dense(1, activation='tanh')(x)
    e = Reshape([-1])(e)

    alpha = Activation('softmax')(e)

    c = Permute([2,1])(RepeatVector(rnn_units)(alpha))
    c = Multiply()([x,c])
    c = Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c)

    notes_out = Dense(n_notes, activation = 'softmax', name = 'pitch')(c)
    durations_out = Dense(n_durations, activation = 'softmax', name='duration')(c)

    print(notes_out)
    model = Model([notes_in, durations_in], [notes_out, durations_out])

    att_model = Model([notes_in, durations_in], alpha)

    opti = RMSprop(learning_rate = 0.001)
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'], optimizer=opti)
    
    return model


#create_network()

In [6]:
def get_notes():

    notes = []
    durations = []
    for file in glob.glob("midi/jazz2/*.mid"):
        try:
            midi = converter.parse(file)
            print("Parsing %s" % file)
            notes_to_parse = None
        #try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except Exception as inst:# file has notes in a flat structure
            notes_to_parse = midi.flat.notes
            print(inst)
            pass
        
        #print(notes_to_parse)
        for element in notes_to_parse:
            #print(element)
            if isinstance(element, note.Note):
                if element.isRest:
                    notes.append(str(element.name))
                    durations.append(element.duration.quarterLength)
                else:
                    notes.append(str(element.pitch))
                    durations.append(element.duration.quarterLength)
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                durations.append(element.duration.quarterLength)

    print("notes", notes)
    pickle.dump(notes, open('notes.p', 'wb'))
    pickle.dump(durations, open('durations.p', 'wb'))
    
    #print("notes: ", notes, "\n")
    return notes, durations


def prepare_sequences(notes, durations, n_vocab, d_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 32 

    pitchnames = sorted(set(item for item in notes))
    #print("pitchnames: ", pitchnames, "\n")
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    #print("note_to_int: ", note_to_int, "\n")
    
    durationnames = sorted(set(item for item in durations))
    #print("pitchnames: ", pitchnames, "\n")
    duration_to_int = dict((note, number) for number, note in enumerate(durationnames))
    #print("note_to_int: ", note_to_int, "\n")

    network_input = []
    network_output = []
    network_input2 = []
    network_output2 = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        #print("sequence_in: ", sequence_in)
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        #print("network_input: ", network_input)
        network_output.append(note_to_int[sequence_out])
        
    for i in range(0, len(durations) - sequence_length, 1):
        sequence_in = durations[i:i + sequence_length]
        #print("sequence_in: ", sequence_in)
        sequence_out = durations[i + sequence_length]
        network_input2.append([duration_to_int[char] for char in sequence_in])
        #print("network_input: ", network_input)
        network_output2.append(duration_to_int[sequence_out])
        
    print("network input", network_input)  
    print("network input", network_input2)
    print("network output", network_output)
    print("network output", network_output2)

    n_patterns = len(network_input)
    n_patterns2 = len(network_input2)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input2 = np.reshape(network_input2, (n_patterns2, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    network_input2 = network_input2 / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)
    network_output2 = np_utils.to_categorical(network_output2)

    return (network_input, network_output, network_input2, network_output2)

In [29]:
def train_network():
    """ Train a Neural Network to generate music """
    notes, durations = get_notes()

    n_vocab = len(set(notes))
    d_vocab = len(set(durations))
    
    
    network_input, network_output, network_input2, network_output2 = prepare_sequences(notes, durations, n_vocab, d_vocab)
    
    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))
    #model = create_network(network_input2, d_vocab)
    model = create_network(notes_in, durations_in, n_vocab, d_vocab)

 
    checkpoint = ModelCheckpoint(
        "weights2-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    ) 
    
    callbacks_list = [checkpoint]

    # Your line of code here
    #model.fit(network_input2, network_output2, epochs=5, callbacks=callbacks_list)
    model.fit([network_input, network_input2], [network_output, network_output2], epochs=5, callbacks=callbacks_list)
    #model.summary()


train_network()
print("Finished Training")

Parsing midi/jazz2/OrnetteColeman_Chronology_FINAL.mid
Parsing midi/jazz2/StanGetz_BodyAndSoul_FINAL.mid
Parsing midi/jazz2/ColemanHawkins_Stompin'AtTheSavoy_FINAL.mid
Parsing midi/jazz2/KennyWheeler_DoubleVision_FINAL.mid
Parsing midi/jazz2/LeeKonitz_BopGoesToLeesel_FINAL.mid
Parsing midi/jazz2/FreddieHubbard_SpeakNoEvil_FINAL.mid
Parsing midi/jazz2/MilesDavis_Walkin'_FINAL.mid
Parsing midi/jazz2/ArtPepper_Stardust-2_FINAL.mid
Parsing midi/jazz2/LeeKonitz_Crosscurrent_FINAL.mid
Parsing midi/jazz2/VonFreeman_PassItOn_FINAL.mid
Parsing midi/jazz2/MilesDavis_SoWhat_FINAL.mid
Parsing midi/jazz2/MiltJackson_AllTheThingsYouAre_FINAL.mid
Parsing midi/jazz2/PaulDesmond_AloneTogether_FINAL.mid
Parsing midi/jazz2/PhilWoods_CrazyRhythm_FINAL.mid
Parsing midi/jazz2/BennyCarter_SweetLorraine_FINAL.mid
Parsing midi/jazz2/DavidLiebman_BeginTheBeguine_FINAL.mid
Parsing midi/jazz2/DavidLiebman_Milestones_FINAL.mid
Parsing midi/jazz2/GerryMulligan_LineForLyons_FINAL.mid
Parsing midi/jazz2/BobBerg_Natur

Parsing midi/jazz2/HerbieHancock_Orbits_FINAL.mid
Parsing midi/jazz2/JohnColtrane_Mr.P.C._FINAL.mid
Parsing midi/jazz2/SteveColeman_TakeTheColtrane_FINAL.mid
Parsing midi/jazz2/WoodyShaw_Imagination_FINAL.mid
Parsing midi/jazz2/JoeLovano_Work_FINAL.mid
Parsing midi/jazz2/KennyGarrett_BrotherHubbard-2_FINAL.mid
Parsing midi/jazz2/BuckClayton_DestinationK.C._FINAL.mid
Parsing midi/jazz2/JoeHenderson_In'NOut-1_FINAL.mid
Parsing midi/jazz2/HerbieHancock_Dolores_FINAL.mid
Parsing midi/jazz2/LeeKonitz_MeanToMe_FINAL.mid
Parsing midi/jazz2/DonEllis_YouSteppedOutOfADream-2_FINAL.mid
Parsing midi/jazz2/DonByas_BodyAndSoul_FINAL.mid
Parsing midi/jazz2/OrnetteColeman_Ramblin'_FINAL.mid
Parsing midi/jazz2/CharlieParker_OutOfNowhere_FINAL.mid
Parsing midi/jazz2/SidneyBechet_LimehouseBlues_FINAL.mid
Parsing midi/jazz2/FatsNavarro_OurDelight_FINAL.mid
Parsing midi/jazz2/SonnyRollins_VierdBlues_FINAL.mid
Parsing midi/jazz2/WarneMarsh_Wow-1_FINAL.mid
Parsing midi/jazz2/DonByas_CognacBlues_FINAL.mid
Par

Parsing midi/jazz2/BobBerg_YouAndTheNightAndTheMusic_FINAL.mid
Parsing midi/jazz2/DonByas_OutOfNowhere_FINAL.mid
Parsing midi/jazz2/LeeKonitz_Marshmallow_FINAL.mid
Parsing midi/jazz2/HarryEdison_DidYouCallHerToday_FINAL.mid
Parsing midi/jazz2/ColemanHawkins_It'sOnlyAPapermoon_FINAL.mid
Parsing midi/jazz2/JohnAbercrombie_Ralph'sPianoWaltz_FINAL.mid
Parsing midi/jazz2/RoyEldridge_St.LouisBlues_FINAL.mid
Parsing midi/jazz2/ChrisPotter_Togo_FINAL.mid
Parsing midi/jazz2/SteveColeman_DoubleVision_FINAL.mid
Parsing midi/jazz2/SonnyStitt_BlueMode_FINAL.mid
Parsing midi/jazz2/RedGarland_Oleo_FINAL.mid
Parsing midi/jazz2/SteveColeman_TheOracle-1_FINAL.mid
Parsing midi/jazz2/BennyGoodman_Runnin'Wild_FINAL.mid
Parsing midi/jazz2/WyntonMarsalis_Caravan_FINAL.mid
Parsing midi/jazz2/ChetBaker_ThereWillNeverBeAnotherYou-2_FINAL.mid
Parsing midi/jazz2/HankMobley_Remember_FINAL.mid
Parsing midi/jazz2/CliffordBrown_I'llRememberApril_AlternateTake2_FINAL.mid
Parsing midi/jazz2/KennyDorham_BluesInBe=Bop_FI

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KerasTensor(type_spec=TensorSpec(shape=(None, 116), dtype=tf.float32, name=None), name='pitch/Softmax:0', description="created by layer 'pitch'")
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this

In [45]:
def prepare_sequences_prediction(notes, pitchnames, durations, durationnames, n_vocab, d_vocab):

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    duration_to_int = dict((dur, number) for number, dur in enumerate(durationnames))
    sequence_length = 32
    network_input = []
    network_input2 = []
    output = []
    output2 = []
    
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])
        
    for i in range(0, len(durations) - sequence_length, 1):
        sequence_in = durations[i:i + sequence_length]
        sequence_out = durations[i + sequence_length]
        network_input2.append([duration_to_int[char] for char in sequence_in])
        output2.append(duration_to_int[sequence_out])
 
    n_patterns = len(network_input)
    n_patterns2 = len(network_input2)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    normalized_input2 = np.reshape(network_input2, (n_patterns2, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)
    normalized_input2 = normalized_input2 / float(n_vocab)

    return (network_input, normalized_input, network_input2, normalized_input2)

def generate_notes(model, network_input, network_input2, pitchnames, durationnames, n_vocab, d_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    import random
    # Starts the melody by picking a random sequence from the input as a starting point
    start = np.random.randint(0, len(network_input)-1)
    start2 = np.random.randint(0, len(network_input2)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    int_to_duration = dict((number, dur) for number, dur in enumerate(durationnames))

    pattern = network_input[start]
    pattern2 = network_input2[start]
    prediction_output = []
    prediction_output2 = []

    for note_index in range(200):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input2 = np.reshape(pattern2, (1, len(pattern2), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction_input2 = prediction_input2 / float(d_vocab)
        
        ### Copy the line below from your above implementation.
        model.summary()
        prediction, prediction2 = model.predict([prediction_input, prediction_input2])
        print(prediction, prediction2)
        
        
        indices = list(np.nonzero(prediction[0])[0])
        indices2 = list(np.nonzero(prediction2[0])[0])
        probs = prediction[0]
        probs2 = prediction2[0]
        index = random.choices(indices, weights=probs, k=1)
        index2 = random.choices(indices2, weights=probs2, k=1)
        index = index[0]
        index2 = index2[0]

#         index = np.argmax(prediction)
#         index2 = np.argmax(prediction2)
        result = int_to_note[index]
        result2 = int_to_duration[index2]
        prediction_output.append(result)
        prediction_output2.append(result2)

        pattern.append(index)
        pattern2.append(index2)
        pattern = pattern[1:len(pattern)]
        pattern2 = pattern2[1:len(pattern2)]

    return prediction_output, prediction_output2

In [46]:
def generate():
    notes = pickle.load(open('notes.p', 'rb'))
    durations = pickle.load(open('durations.p', 'rb'))
    
    pitchnames = sorted(set(item for item in notes))
    durationnames = sorted(set(item for item in durations))
    
    n_vocab = len(set(notes))
    d_vocab = len(set(durations))

    network_input, normalized_input, network_input2, normalized_input2 = prepare_sequences_prediction(notes, pitchnames, durations, durationnames, n_vocab, d_vocab)
    
    notes_in = Input(shape = (None,))
    durations_in = Input(shape = (None,))
    model = create_network(notes_in, durations_in, n_vocab, d_vocab)
    
    ### Add a line to load the weights here
    model.load_weights("weights2-improvement-02-5.0117-bigger.hdf5")
    
    

    prediction_output, prediction_output2 = generate_notes(model, network_input, network_input2, pitchnames, durationnames, n_vocab, d_vocab)
    create_midi(prediction_output, prediction_output2)

In [49]:
def create_midi(prediction_output, prediction_output2):
    offset = 0
    output_notes = []
    for pattern, pattern2 in zip(prediction_output, prediction_output2):
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Saxophone()
                notes.append(new_note)
            print("patern2:", pattern2)
            #d = duration.quarterLength(pattern2)
            new_chord = chord.Chord(notes, quarterLength=pattern2)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            #d = duration.quarterLength(pattern2)
            new_note = note.Note(pattern, quarterLength=pattern2)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Saxophone()
            output_notes.append(new_note)
        offset += 0.5
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output.mid')
    
generate()
print("done")

KerasTensor(type_spec=TensorSpec(shape=(None, 116), dtype=tf.float32, name=None), name='pitch/Softmax:0', description="created by layer 'pitch'")
Model: "model_32"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_34 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_34 (Embedding)        (None, None, 100)    11600       input_33[0][0]                   
__________________________________________________________________________________________________
embedding_35 (Embedding)        (None, None,

[[4.93174548e-06 2.38181792e-06 1.90461492e-06 2.61795412e-06
  3.85672593e-06 1.41135672e-06 1.45185516e-06 1.65228391e-06
  6.41106226e-06 1.44637022e-06 1.62972196e-06 4.89595732e-06
  1.38750534e-06 2.28408885e-06 2.11115002e-06 3.08150788e-06
  1.68574672e-06 2.87562239e-06 2.48220340e-06 1.99166311e-06
  1.93721871e-06 1.39457347e-06 1.31526940e-06 7.39331608e-06
  1.84727344e-06 1.80102074e-06 1.45663444e-06 1.51913594e-06
  1.29337980e-06 1.53074416e-06 2.34432559e-06 2.83139889e-06
  2.14938927e-06 2.94995357e-06 1.38844109e-06 1.85550311e-06
  1.36738470e-06 3.29457225e-06 2.04454636e-06 6.36602590e-06
  3.32417608e-06 3.58280931e-06 3.70100588e-06 1.77406594e-06
  2.50901644e-06 2.40011855e-06 1.50151197e-06 1.45613160e-06
  3.39300959e-06 2.76081323e-06 4.52610311e-06 1.16377277e-03
  2.41956767e-02 4.95761409e-02 6.58461452e-03 1.46250704e-06
  2.24498496e-03 3.93613204e-02 5.77251352e-02 7.56152533e-03
  1.57238594e-06 1.16870087e-03 2.18414105e-02 3.20853703e-02
  4.5647

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[[4.93174548e-06 2.38181792e-06 1.90461492e-06 2.61795412e-06
  3.85672593e-06 1.41135672e-06 1.45185516e-06 1.65228391e-06
  6.41106226e-06 1.44637022e-06 1.62972196e-06 4.89595732e-06
  1.38750534e-06 2.28408885e-06 2.11115002e-06 3.08150788e-06
  1.68574672e-06 2.87562239e-06 2.48220340e-06 1.99166311e-06
  1.93721871e-06 1.39457347e-06 1.31526940e-06 7.39331608e-06
  1.84

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set